In [ ]:
# STEP 0: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# STEP 1: Install necessary libraries
!pip install transformers datasets sentence-transformers faiss-cpu --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.7 MB/s eta 0:00:00


In [ ]:
# STEP 2: Load and process the verified CSV knowledge base
import pandas as pd

csv_path = '/content/drive/MyDrive/skin_disease_knowledge_base.csv'
df = pd.read_csv(csv_path)
# Create knowledge chunks
knowledge_texts = df.apply(
    lambda row: f"Disease: {row['Disease Name']}. Info: {row['Description']} Symptoms: {row['Symptoms']} Treatment: {row['Treatment']}",
    axis=1
).tolist()

In [ ]:
# STEP 3: Setup sentence-transformer + FAISS index
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = embedder.encode(knowledge_texts, convert_to_numpy=True)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# STEP 4: Load your fine-tuned RoBERTa model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model_path = '/content/drive/MyDrive/Fine-tuned Models Weights/RoBERTa/RoBERTa'  # <-- Make sure this path is correct
tokenizer = AutoTokenizer.from_pretrained(model_path)
finetuned_model = AutoModelForSequenceClassification.from_pretrained(model_path)

# STEP 5: Define label map
label_map = {
    0: 'Vitiligo',
    1: 'Folliculitis',
    2: 'Eczema',
    3: 'Ringworm (tinea corporis)',
    4: "Athlete's foot (tinea pedis)",
    5: 'Rosacea',
    6: 'Psoriasis',
    7: 'Shingles',
    8: 'Impetigo',
    9: 'Scabies',
    10: 'Contact Dermatitis',
    11: 'Acne',
    12: 'Lupus',
    13: 'Seborrheic dermatitis',
    14: 'Milia'
}


In [ ]:
def extract_symptom_line(text):
    # Returns the 'Symptoms: ...' portion from the retrieved info
    for part in text.split("Symptoms:"):
        if part.strip():
            return part.strip().split("Treatment:")[0].strip()
    return ""

def query_bot(user_input):
    # FAISS retrieval
    query_embedding = embedder.encode([user_input], convert_to_numpy=True)
    D, I = index.search(query_embedding, k=1)
    retrieved_text = knowledge_texts[I[0][0]]
    retrieved_symptoms = extract_symptom_line(retrieved_text)

    # Combine user symptoms + retrieved symptoms
    combined_input = f"{user_input.strip()} Also noticing: {retrieved_symptoms}"

    # RoBERTa classification
    inputs = tokenizer(combined_input, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = finetuned_model(**inputs)
        predicted_class_id = torch.argmax(outputs.logits, dim=1).item()

    predicted_disease = label_map.get(predicted_class_id, "Unknown")
    return predicted_disease, retrieved_text


In [ ]:
# STEP 7: Runtime interaction loop
while True:
    user_input = input("Enter your symptoms (or type 'exit' to stop): ")
    if user_input.lower() == 'exit':
        break
    diagnosis, context = query_bot(user_input)
    print(f"\nDiagnosis: {diagnosis}")
    print(f"Retrieved Info: {context}\n")



Diagnosis: Contact Dermatitis
Retrieved Info: Disease: Contact Dermatitis. Info: Contact dermatitis is a skin reaction resulting from exposure to allergens or irritants. Symptoms: Red rash; itching; dry, cracked, or scaly skin; blisters; burning or stinging sensations. Treatment: Identifying and avoiding triggers; topical corticosteroids; moisturizers; antihistamines.


Diagnosis: Psoriasis
Retrieved Info: Disease: Eczema. Info: Eczema, or atopic dermatitis, is a chronic skin condition characterized by dry, itchy, and inflamed skin. Symptoms: Dry, sensitive skin; intense itching; red or brownish-gray patches; thickened, cracked, or scaly skin; possible oozing and crusting. Treatment: Moisturizers, topical corticosteroids, antihistamines, and avoiding triggers.


Diagnosis: Psoriasis
Retrieved Info: Disease: Eczema. Info: Eczema, or atopic dermatitis, is a chronic skin condition characterized by dry, itchy, and inflamed skin. Symptoms: Dry, sensitive skin; intense itching; red or brown